In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
import jdatetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, 
    r2_score, mean_absolute_percentage_error
)
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
from tensorflow import keras

In [16]:
df = pd.read_csv('../data/processed/main_feats.csv')
df.head()

,start_date,end_date,project_duration,x1,x2,x3,x5,x51,y
0,1396-01-01,1400-01-01,48,3150.0,920.0,1,1703.672462,16,9979.795043
1,1395-01-01,1400-10-01,69,7600.0,1140.0,1,1446.077707,23,16362.045147
2,1398-01-01,1401-10-01,45,4800.0,840.0,1,2376.845399,15,14120.093280
3,1397-04-01,1398-04-01,12,685.0,202.0,1,1207.741141,4,2560.672748
4,1387-01-01,1390-04-01,39,3000.0,800.0,1,410.000000,13,700.000000


In [17]:
df_economics = pd.read_csv('../data/processed/cubic_imputated_economics.csv')
df_economics.head()

,Date,b1,b2,b3,b4,b5,b6,b7,b8,b9,...,Sin b6,Sin b7,Sin b8,Sin b9,Sin b10,Sin b14,Sin b17,Sin b18,Sin B-23,Sin B-24
0,1360-01-01,-0.312302,-0.573053,-1.008179,-0.486658,-0.519259,-0.550486,-0.524004,-0.474211,-0.763966,...,-0.760378,-0.519490,-0.536372,-0.881368,-0.592062,-0.674918,-0.894974,-0.765731,-0.739974,-0.624255
1,1360-04-01,-0.526017,-0.573053,-1.045895,-0.486639,-0.519352,-0.550486,-0.523908,-0.474211,-0.763704,...,-0.760378,-0.519405,-0.536372,-0.881702,-0.592062,-0.674918,-0.894543,-0.765731,-0.739974,-0.624255
2,1360-07-01,-0.431611,-0.573053,-1.097080,-0.486628,-0.519297,-0.550486,-0.523908,-0.474146,-0.763828,...,-0.760378,-0.519405,-0.536306,-0.881544,-0.592062,-0.674918,-0.894332,-0.765731,-0.739974,-0.624255
3,1360-10-01,-0.573509,-0.573053,-1.083610,-0.486563,-0.519403,-0.550486,-0.523716,-0.474211,-0.763741,...,-0.760378,-0.519234,-0.536372,-0.881655,-0.592062,-0.674918,-0.893711,-0.765731,-0.739974,-0.624255
4,1361-01-01,-0.586830,-0.573053,-1.038711,-0.486562,-0.518833,-0.550486,-0.523716,-0.474015,-0.763624,...,-0.760378,-0.519234,-0.536175,-0.881803,-0.590450,-0.674918,-0.894031,-0.765731,-0.739974,-0.624255


In [18]:
# filter df_economics drop extera rows for now (TODO: Add seasonal and  lag later)
df_economics = df_economics[df_economics['Date'].isin(df['start_date'])]
df = pd.merge(df, df_economics, how='left', left_on='start_date', right_on='Date')
del df_economics
df = df[[c for c in df if c not in ['y']] + ['y']]
# drop Date column
df = df.drop(['Date'], axis=1)
print(df.shape)
df.head()

(400, 54)


,start_date,end_date,project_duration,x1,x2,x3,x5,x51,b1,b2,...,Sin b7,Sin b8,Sin b9,Sin b10,Sin b14,Sin b17,Sin b18,Sin B-23,Sin B-24,y
0,1396-01-01,1400-01-01,48,3150.0,920.0,1,1703.672462,16,-0.627952,0.512964,...,0.473973,0.263595,-9.090529e-01,0.294251,-0.124358,0.975063,0.964070,0.954525,0.769512,9979.795043
1,1395-01-01,1400-10-01,69,7600.0,1140.0,1,1446.077707,23,-0.903059,0.432169,...,0.383201,0.210828,-6.687348e-01,0.225918,-0.221708,0.999795,0.905790,0.901170,0.658368,16362.045147
2,1398-01-01,1401-10-01,45,4800.0,840.0,1,2376.845399,15,-0.660964,1.126800,...,0.929428,0.999566,-1.262291e-01,0.307541,0.975338,-0.041006,0.865552,0.931351,0.889746,14120.093280
3,1397-04-01,1398-04-01,12,685.0,202.0,1,1207.741141,4,-0.268284,0.718625,...,0.653622,0.586774,-2.449294e-16,0.756817,0.921560,0.433860,0.996015,0.999972,0.954203,2560.672748
4,1387-01-01,1390-04-01,39,3000.0,800.0,1,410.000000,13,1.995128,-0.328568,...,-0.258078,-0.278387,-2.820141e-01,-0.397584,-0.609905,-0.503466,-0.134573,-0.134950,-0.311864,700.000000


In [19]:
df = df[[c for c in df if 'Sin' not in c]]
df = df[[c for c in df if 'Cosine' not in c]]
print(df.shape)
df.head()

(400, 24)


,start_date,end_date,project_duration,x1,x2,x3,x5,x51,b1,b2,...,b7,b8,b9,b10,b14,b17,b18,B-23,B-24,y
0,1396-01-01,1400-01-01,48,3150.0,920.0,1,1703.672462,16,-0.627952,0.512964,...,0.473683,0.223436,1.586044,0.212582,-0.078869,1.076990,0.977018,0.980000,0.686014,9979.795043
1,1395-01-01,1400-10-01,69,7600.0,1140.0,1,1446.077707,23,-0.903059,0.432169,...,0.377241,0.177932,1.434758,0.162199,-0.141420,0.930535,0.850433,0.867476,0.561457,16362.045147
2,1398-01-01,1401-10-01,45,4800.0,840.0,1,2376.845399,15,-0.660964,1.126800,...,1.144272,1.340417,2.280080,2.013554,1.134411,1.909989,1.572436,1.501990,1.597541,14120.093280
3,1397-04-01,1398-04-01,12,685.0,202.0,1,1207.741141,4,-0.268284,0.718625,...,0.683344,0.525254,2.326954,1.625061,0.741419,1.616050,1.245815,1.208175,0.989856,2560.672748
4,1387-01-01,1390-04-01,39,3000.0,800.0,1,410.000000,13,1.995128,-0.328568,...,-0.250400,-0.236309,-0.105879,-0.291025,-0.414927,-0.316633,-0.101297,-0.104613,-0.247783,700.000000


In [20]:
# convert start_date and end_date to jdatetime
df['start_date'] = df['start_date'].apply(lambda x: jdatetime.datetime.strptime(x, '%Y-%m-%d'))
df['end_date'] = df['end_date'].apply(lambda x: jdatetime.datetime.strptime(x, '%Y-%m-%d'))
# convert start_date and end_date to gregorian
df['start_date'] = df['start_date'].apply(lambda x: x.togregorian())
df['end_date'] = df['end_date'].apply(lambda x: x.togregorian())
df.head()

,start_date,end_date,project_duration,x1,x2,x3,x5,x51,b1,b2,...,b7,b8,b9,b10,b14,b17,b18,B-23,B-24,y
0,2017-03-21,2021-03-21,48,3150.0,920.0,1,1703.672462,16,-0.627952,0.512964,...,0.473683,0.223436,1.586044,0.212582,-0.078869,1.076990,0.977018,0.980000,0.686014,9979.795043
1,2016-03-20,2021-12-22,69,7600.0,1140.0,1,1446.077707,23,-0.903059,0.432169,...,0.377241,0.177932,1.434758,0.162199,-0.141420,0.930535,0.850433,0.867476,0.561457,16362.045147
2,2019-03-21,2022-12-22,45,4800.0,840.0,1,2376.845399,15,-0.660964,1.126800,...,1.144272,1.340417,2.280080,2.013554,1.134411,1.909989,1.572436,1.501990,1.597541,14120.093280
3,2018-06-22,2019-06-22,12,685.0,202.0,1,1207.741141,4,-0.268284,0.718625,...,0.683344,0.525254,2.326954,1.625061,0.741419,1.616050,1.245815,1.208175,0.989856,2560.672748
4,2008-03-20,2011-06-22,39,3000.0,800.0,1,410.000000,13,1.995128,-0.328568,...,-0.250400,-0.236309,-0.105879,-0.291025,-0.414927,-0.316633,-0.101297,-0.104613,-0.247783,700.000000


In [21]:
# convert start_date and end_date to UNIX timestamp
df['start_date'] = pd.to_datetime(df['start_date']).astype(np.int64) // 10**9
df['end_date'] = pd.to_datetime(df['end_date']).astype(np.int64) // 10**9
print(df.shape)
df.head()

(400, 24)


,start_date,end_date,project_duration,x1,x2,x3,x5,x51,b1,b2,...,b7,b8,b9,b10,b14,b17,b18,B-23,B-24,y
0,1490054400,1616284800,48,3150.0,920.0,1,1703.672462,16,-0.627952,0.512964,...,0.473683,0.223436,1.586044,0.212582,-0.078869,1.076990,0.977018,0.980000,0.686014,9979.795043
1,1458432000,1640131200,69,7600.0,1140.0,1,1446.077707,23,-0.903059,0.432169,...,0.377241,0.177932,1.434758,0.162199,-0.141420,0.930535,0.850433,0.867476,0.561457,16362.045147
2,1553126400,1671667200,45,4800.0,840.0,1,2376.845399,15,-0.660964,1.126800,...,1.144272,1.340417,2.280080,2.013554,1.134411,1.909989,1.572436,1.501990,1.597541,14120.093280
3,1529625600,1561161600,12,685.0,202.0,1,1207.741141,4,-0.268284,0.718625,...,0.683344,0.525254,2.326954,1.625061,0.741419,1.616050,1.245815,1.208175,0.989856,2560.672748
4,1205971200,1308700800,39,3000.0,800.0,1,410.000000,13,1.995128,-0.328568,...,-0.250400,-0.236309,-0.105879,-0.291025,-0.414927,-0.316633,-0.101297,-0.104613,-0.247783,700.000000


In [22]:
df.dtypes

start_date            int64
end_date              int64
project_duration      int64
x1                  float64
x2                  float64
x3                    int64
x5                  float64
x51                   int64
b1                  float64
b2                  float64
b3                  float64
b4                  float64
b5                  float64
b6                  float64
b7                  float64
b8                  float64
b9                  float64
b10                 float64
b14                 float64
b17                 float64
b18                 float64
B-23                float64
B-24                float64
y                   float64
dtype: object

In [23]:
# start_date, end_date, project_duration, x3, x51 to float
df['start_date'] = df['start_date'].astype(np.float64)
df['end_date'] = df['end_date'].astype(np.float64)
df['project_duration'] = df['project_duration'].astype(np.float64)
df['x3'] = df['x3'].astype(np.float64)
df['x51'] = df['x51'].astype(np.float64)
df.dtypes

start_date          float64
end_date            float64
project_duration    float64
x1                  float64
x2                  float64
x3                  float64
x5                  float64
x51                 float64
b1                  float64
b2                  float64
b3                  float64
b4                  float64
b5                  float64
b6                  float64
b7                  float64
b8                  float64
b9                  float64
b10                 float64
b14                 float64
b17                 float64
b18                 float64
B-23                float64
B-24                float64
y                   float64
dtype: object

In [24]:
# split data to train and test
x_train, x_test, y_train, y_test = train_test_split(
    df[[c for c in df if c not in ['y']]], df['y'], test_size=0.2, random_state=42
)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(320, 23) (80, 23) (320,) (80,)


In [25]:
# Create a nerual network with 1 GRU and 1 drop out
model = keras.Sequential()
model.add(keras.layers.GRU(128, input_shape=(df.shape[1]-1, 1), return_sequences=True))
# model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.GRU(64, return_sequences=False))
# model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 23, 128)           50304     
                                                                 
 gru_3 (GRU)                 (None, 64)                37248     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 87,617
Trainable params: 87,617
Non-trainable params: 0
_________________________________________________________________


2023-07-16 22:13:08.752122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 22:13:08.753018: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 22:13:08.753686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [26]:
# fit model
history = model.fit(
    x_train.values.reshape(x_train.shape[0], x_train.shape[1], 1), y_train,
    epochs=100, batch_size=32, validation_split=0.2, verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto',
            baseline=None, restore_best_weights=True
        )
    ]
)

Epoch 1/100


2023-07-16 22:13:09.040702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 22:13:09.041739: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 22:13:09.043084: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

6/8 [=====================>........] - ETA: 0s - loss: 44400076.0000

2023-07-16 22:13:10.589259: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 22:13:10.590557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 22:13:10.591412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

8/8 [==============================] - 2s 71ms/step - loss: 40401700.0000 - val_loss: 47974512.0000
Epoch 2/100
8/8 [==============================] - 0s 12ms/step - loss: 40353784.0000 - val_loss: 47907424.0000
Epoch 3/100
8/8 [==============================] - 0s 12ms/step - loss: 40317272.0000 - val_loss: 47894152.0000
Epoch 4/100
8/8 [==============================] - 0s 12ms/step - loss: 40309832.0000 - val_loss: 47887940.0000
Epoch 5/100
8/8 [==============================] - 0s 12ms/step - loss: 40304424.0000 - val_loss: 47881912.0000
Epoch 6/100
8/8 [==============================] - 0s 15ms/step - loss: 40299388.0000 - val_loss: 47876512.0000
Epoch 7/100
8/8 [==============================] - 0s 13ms/step - loss: 40294548.0000 - val_loss: 47871136.0000
Epoch 8/100
8/8 [==============================] - 0s 14ms/step - loss: 40289552.0000 - val_loss: 47865936.0000
Epoch 9/100
8/8 [==============================] - 0s 12ms/step - loss: 40284928.0000 - val_loss: 47860752.0000
Epoc

In [27]:
# plot loss
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='train'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='validation'))
fig.update_layout(title='Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

In [28]:
# evaluate model
y_pred = model.predict(x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1))
print('MSE:', mean_squared_error(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))

2023-07-16 22:13:20.297035: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 22:13:20.298038: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 22:13:20.298768: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3/3 [==============================] - 0s 4ms/step
MSE: 26584682.877065472
MAE: 3973.231064874143
R2: -1.4619737782689772
MAPE: 0.9679333225846165
